In [1]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
from pywintypes import com_error
from scripts.tools.issuetracker import IssuetrackerAPI
from os.path import join as pjoin
import re

In [2]:
trace_path = "C:\\Users\\Nathan\\Documents\\PBS\\SW test\\3.0 User Test\\Traceability"
file1 = "Tests 161111 1.xlsx"

In [3]:
xl = Excel(new=False)
wb = xl.Workbooks.Open(pjoin(trace_path, file1))

In [4]:
ws = wb.Worksheets("Sheet1")
cells = ws.Cells
cr = cells.Range
id_start = cells.Find("ID_TEST").Offset(2, 1)
id_end = id_start.End(xlc.xlDown)
frs_start = cells.Find("List Web FRS").Offset(2, 1)
frs_end = frs_start.Offset(id_end.Row - id_start.Row + 1, 1)
test_data = cr(id_start, frs_end).Value

In [5]:
api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
issues = api.download_issues("pbssoftware")

In [6]:
relevant = []
for v in issues.values():
    if v.sprint_milestone == "3.0" and \
        v.tracker == "Specification" and \
        v.status not in ("Closed", "Rejected"):
            relevant.append(v) 


In [17]:
webfrs_re = re.compile(r"(3\.0WebFRS\d\d\d)")
for i in relevant:
    for f in webfrs_re.findall(i.description):
        print(f)

3.0WebFRS121
3.0WebFRS119
3.0WebFRS001
3.0WebFRS002


In [7]:
test_map = []
test_map2 = {}
id_strings = set()
frs_strings = set()
for row in test_data:
    id_test = row[0]
    iid_test = int(id_test)
    if iid_test == id_test:
        id_test = iid_test
    list_web = row[-1] or ""
    frs = list_web.split("\n")
    id_strings.add(id_test)
    for f in filter(None, frs):
        f=f.strip()
        frs_strings.add(f)
        test_map.append((f, id_test))
        flist = test_map2.get(f, None)
        if flist is None:
            flist = test_map2[f] = []
        flist.append(id_test)

In [8]:
def frs_nums(s):
    pos = s.find("FRS")
    num = s[pos+3:]
    return num.split(".")

def num_tuple(s):
    return tuple(int(i) for i in frs_nums(s))
    
def sort_key(t):
    return num_tuple(t[0])
     
test_map.sort(key=sort_key)

In [18]:
def sort_frs_item(s):
    return num_tuple(s)

toplevel_match = re.compile(r"^\>?[\+\*]{2}(FRS\d+)[\+\*]{2}.*$").match
subitem_match = re.compile(r"^\>?[\+\*]+\s\*(FRS[\d\.]+)\:?\*\:?.*$").match
all_frs = set()
for v in relevant:
    lines = v.description.splitlines()
    for line in lines:
        for match in (toplevel_match, subitem_match):
            m = match(line)
            if m:
                frs = m.group(1)
                all_frs.add(frs)
for i in range(1, 121 + 1):
    all_frs.add("3.0WebFRS%03d" % i)
all_frs2 = sorted(all_frs, key=sort_frs_item)
all_frs2[:10]

['3.0WebFRS001',
 '3.0WebFRS002',
 '3.0WebFRS003',
 '3.0WebFRS004',
 '3.0WebFRS005',
 '3.0WebFRS006',
 '3.0WebFRS007',
 '3.0WebFRS008',
 '3.0WebFRS009',
 '3.0WebFRS010']

In [19]:
diff = frs_strings - all_frs
bad_tests = {}
for d in diff:
    tests = test_map2.get(d)
    for t in tests:
        bad = bad_tests.get(t, None)
        if bad is None:
            bad_tests[t] = bad = []
        bad.append(d)
#     print(test_map2.get(d), d)
for t, frs in sorted(bad_tests.items()):
    print(t, frs)
if not bad_tests:
    print("No Bad Tests Found")
else:
    assert False, "Bad Tests found"

No Bad Tests Found


In [20]:
def dump(node, level=0):
    for k, v in sorted(node.children.items()):
        print(" "*level + str(v.id)+ " "+str(v._tests))
        dump(v, level+1)

class Node():
    def __init__(self, id, parent):
        self.id = id
        self.parent = parent
        self.children = {}
        self._tests = []
        
    def add_test(self, id_test):
        self._tests.append(id_test)
        for c in self.children.values():
            c.add_test(id_test)
        
    def get_tests(self):
        return sorted(self._tests)
        
    def get(self, id):
        return self.children.get(id, None)
        
    def add_child(self, id):
        child = self.mk_child(id)
        self.children[id] = child
        return child
    
    def mk_child(self, id):
        return self.__class__("%s.%s"%(self.id, id), self)
    
    def iter(self):
        # use .items() instead of .values() to sort
        for _, v in sorted(self.children.items()):
            yield v
            yield from v.iter()
            
    def __repr__(self):
        return "%s(%r)" % (self.__class__.__name__, self.id)
    
    def is_tested(self):
        if self._tests:
            return True
        elif not self.children:
            return False
        else:
            for c in self.children.values():
                if not c.is_tested():
                    return False
            return True
        
    
frs_match = re.compile(r"(.*FRS\d+)\.?([\d\.]*)").match
    
class Root(Node):
    def __init__(self):
        super().__init__("", None)
        
    def mk_child(self, id):
        return Node(id, self)
        
    def add(self, key):
        frs, nums = frs_match(key).groups()
        child = self.get(frs)
        if not child:
            child = self.add_child(frs)
        if not nums:
            return child
        path = [int(i) for i in nums.split(".")]
        path.reverse()
        node = child
        while path:
            id = path.pop()
            child = node.get(id)
            if child is None:
                child = node.add_child(id)
            node = child
        return node
    
    def lookup(self, key):
        frs, nums = frs_match(key).groups()
        node = self.get(frs)
        if not node or not nums:
            return node
        path = [int(i) for i in nums.split(".")][::-1]
        while path and node:
            id = path.pop()
            node = node.get(id)
        return node
        
        
root_reqs = Root()
for f in all_frs:
    node = root_reqs.add(f)
for f in all_frs:
    tests = test_map2.get(f, ())
    node = root_reqs.lookup(f)
    assert node, f
    for t in tests:
        node.add_test(t)
    
    
#dump(root_reqs)
ids = [v.id for v in root_reqs.iter()]
set_ids = set(ids)
set_allfrs = set(all_frs)
assert set_ids == set_allfrs
for id in set_ids:
    assert root_reqs.lookup(id).id == id

In [21]:
seen_tests = set()
for v in root_reqs.iter():
    for t in v.get_tests():
        seen_tests.add(t)
set_map2 = set()
for v in test_map2.values():
    for t in v:
        set_map2.add(t)
diff = seen_tests - set_map2
assert not diff, diff
print("Seen tests success")

Seen tests success


In [30]:
paste_data = []
for f in all_frs2:
    node = root_reqs.lookup(f)
    if not node:
        tests = ()
        tested = ""
    else:
        tests = "\n".join(str(i) for i in node.get_tests())
        tested = "Y" if node.is_tested() else ""
    paste_data.append((f, f.count("."), tests, tested))

In [40]:
try:
    ws2 = wb.Worksheets("Matrix")
    # clear twice to clear autofilter
    ws2.UsedRange.Clear()
    ws2.UsedRange.Clear()
except com_error:
    ws2 = wb.Worksheets.Add()
    ws2.Name = "Matrix"

cells2 = ws2.Cells
cr2 = cells2.Range
    
headers = [("FRS Number", "Level", "id_test", "Tested")]
cr2(cr2("A1"), cr2("A1").Offset(1, len(headers[0]))).Value = headers

frs_start = cr2("A2")
frs_end = frs_start.Offset(len(paste_data), 1)

id_start = frs_start.Offset(1, 2)
id_end = frs_end.Offset(1, 2)

tested_start = id_start.Offset(1,2)
tested_end = id_end.Offset(1,2)

paste_start = frs_start
paste_end = frs_end.Offset(1, len(paste_data[0]))

In [41]:
with screen_lock(xl):
    paste_range = cr2(paste_start, paste_end)
    paste_range.Value = paste_data

    cr2(frs_start, frs_end).VerticalAlignment = xlc.xlTop
    cr2(id_start, id_end).HorizontalAlignment = xlc.xlRight
    cr2(tested_start, tested_end).HorizontalAlignment = xlc.xlRight
    cr2(tested_start, tested_end).VerticalAlignment = xlc.xlTop
    
    col_os = paste_end.Column - paste_start.Column + 1
    for offset, (item, level, _, tested) in enumerate(paste_data, 1):
        if not tested:
            row = cr2(paste_start.Offset(offset, 1), paste_start.Offset(offset, col_os))
            row.Interior.Pattern = xlc.xlSolid
            row.Interior.PatternColorIndex = xlc.xlAutomatic
            row.Interior.ThemeColor = xlc.xlThemeColorAccent2
            row.Interior.TintAndShade = 0.399975585192419
            row.Interior.PatternTintAndShade = 0
            row[1].IndentLevel = level
 
    cr2(paste_start.Offset(0, 1), paste_end).AutoFilter(Field=col_os, Criteria1="=")
    
    # fit after filter to account for width of filter icon
    for c in paste_range.Columns:
        c.EntireColumn.AutoFit()
    

FRS2819
FRS2320
FRS2769
FRS3288
FRS2884
FRS2906
FRS2871
FRS1073
FRS3176
FRS2877
FRS2796
FRS2828
FRS2905
FRS2891
FRS1902
FRS2851
FRS2888
FRS2907
FRS3226
FRS2875
FRS2304
FRS2321
FRS3225
FRS420
FRS2908
FRS2926
FRS2880
FRS1583
FRS2367
FRS2865
FRS2876
FRS1759
FRS2881
FRS2860
FRS3010
FRS3289
FRS2856
